In [22]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from tasks import NER
from utils_ner import TokenClassificationDataset
import torch
import  nltk

In [23]:
# load phobert-ner đã được fine-tuned
phobert_ner = AutoModelForTokenClassification.from_pretrained("huyenbui117/Covid19_phoNER")
# load tokenizer của phobert
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base", use_fast=False)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [24]:
labels = NER().get_labels(None)
labels

['O',
 'B-ORGANIZATION',
 'O-ORGANIZATION',
 'I-ORGANIZATION',
 'B-LOCATION',
 'I-LOCATION',
 'B-SYMPTOM_AND_DISEASE',
 'I-SYMPTOM_AND_DISEASE',
 'B-DATE',
 'I-DATE',
 'B-PATIENT_ID',
 'I-PATIENT_ID',
 'B-AGE',
 'I-AGE',
 'B-NAME',
 'I-NAME',
 'B-JOB',
 'I-JOB',
 'B-TRANSPORTATION',
 'I-TRANSPORTATION',
 'B-GENDER']

In [43]:
from vncorenlp import VnCoreNLP
rdrsegmenter = VnCoreNLP("vncorenlp/VnCoreNLP-1.1.1.jar", annotators="wseg", max_heap_size='-Xmx500m')

In [53]:
with open("sentences.txt", 'r',encoding="utf-8") as file:
    for line in file:
        sentences = rdrsegmenter.tokenize(line)
        for sentence in sentences:
            sequence = " ".join(sentence)  # tạo câu mới đã được tokenized

            input_ids = torch.tensor([tokenizer.encode(sequence)])  # lấy id của các tokens tương ứng
            # không dùng tokenize(decode(encode)), text sẽ bị lỗi khi tokenize do conflict với tokenizer mặc định
            tokens = tokenizer.convert_ids_to_tokens(tokenizer.encode(sequence))  # lấy các token để đánh tags

            outputs = phobert_ner(input_ids).logits
            predictions = torch.argmax(outputs, dim=2)
            print(predictions)


            for i in [(token, labels[prediction]) for token, prediction in zip(tokens, predictions[0].numpy())]:
                print(i)
            print('---------------------------------------------')

tensor([[ 0,  0,  0,  0, 10, 10, 10,  0,  0, 14, 14,  0,  0,  0,  4,  5,  5,  0,
          4,  5,  5,  5,  0,  4,  5,  0,  4,  5,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  4,  5,  5,  5,  5,  5,  0,  0,  0,  0,  0,  0,  8,  9,  9,  0,  0]])
('<s>', 'O')
('Bệnh', 'O')
('nhân', 'O')
('số', 'O')
('94@@', 'B-PATIENT_ID')
('64@@', 'B-PATIENT_ID')
('01', 'B-PATIENT_ID')
('tên', 'O')
('là', 'O')
('NH@@', 'B-NAME')
('T', 'B-NAME')
('thường', 'O')
('trú', 'O')
('tại', 'O')
('thôn', 'B-LOCATION')
('Thành', 'I-LOCATION')
('Trung', 'I-LOCATION')
(',', 'O')
('xã', 'B-LOCATION')
('Ph@@', 'I-LOCATION')
('ồn_@@', 'I-LOCATION')
('Xương', 'I-LOCATION')
(',', 'O')
('huyện', 'B-LOCATION')
('Yên_Thế', 'I-LOCATION')
(',', 'O')
('tỉnh', 'B-LOCATION')
('Bắc_Giang', 'I-LOCATION')
(',', 'O')
('thực_hiện', 'O')
('test', 'O')
('nhanh', 'O')
('CO@@', 'O')
('VI@@', 'O')
('D-@@', 'O')
('19', 'O')
('tại', 'O')
('Trung', 'B-LOCATION')
('tâm', 'I-LOCATION')
('Y', 'I-LOCATION')
('tế', 'I-LOCATION')
('huyện', 'I-LOCA